In [1]:
%load_ext autoreload
%autoreload 2

import torch
import json

from models import z_yolo
from data_process.waymo_dataloader import create_train_dataloader

In [2]:
configs = {"dataset_dir": "../../Data/waymo_data/", "pin_memory": True, 
           "batch_size": 4, "num_workers": 4, "device": 'cpu'}

train_loader = create_train_dataloader(configs)

In [3]:
model = z_yolo.Zolo()

for batch_idx, batch_data in enumerate(train_loader):
#     print(batch_data['bev'])
    outputs = model(batch_data['bev'].float())
    outputs
    print(batch_idx, outputs['class_type'].size())
    

0 torch.Size([4, 3, 400, 400])
1 torch.Size([4, 3, 400, 400])
2 torch.Size([4, 3, 400, 400])
3 torch.Size([4, 3, 400, 400])
4 torch.Size([4, 3, 400, 400])
5 torch.Size([4, 3, 400, 400])
6 torch.Size([4, 3, 400, 400])
7 torch.Size([4, 3, 400, 400])
8 torch.Size([4, 3, 400, 400])
9 torch.Size([4, 3, 400, 400])
10 torch.Size([4, 3, 400, 400])
11 torch.Size([4, 3, 400, 400])
12 torch.Size([4, 3, 400, 400])
13 torch.Size([4, 3, 400, 400])
14 torch.Size([4, 3, 400, 400])
15 torch.Size([4, 3, 400, 400])
16 torch.Size([4, 3, 400, 400])
17 torch.Size([4, 3, 400, 400])
18 torch.Size([4, 3, 400, 400])
19 torch.Size([4, 3, 400, 400])
20 torch.Size([4, 3, 400, 400])
21 torch.Size([4, 3, 400, 400])
22 torch.Size([4, 3, 400, 400])
23 torch.Size([4, 3, 400, 400])
24 torch.Size([4, 3, 400, 400])
25 torch.Size([4, 3, 400, 400])
26 torch.Size([4, 3, 400, 400])
27 torch.Size([4, 3, 400, 400])
28 torch.Size([4, 3, 400, 400])
29 torch.Size([4, 3, 400, 400])
30 torch.Size([4, 3, 400, 400])
31 torch.Size([4, 

In [ ]:
batch_data['bev'].float()

In [ ]:
x = torch.load("../../Data/waymo_data/pt12/segment-6791933003490312185_2607_000_2627_000_with_camera_labels0.pt", map_location=torch.device('cpu'))

In [ ]:
x

In [ ]:
model.eval()

In [ ]:
with open("../cfg/cfg.json") as f:
    configs = json.load(f)

In [ ]:
configs

### 1. Model

Try fpn_resnet_18 for default.

* model:      fpn_resnet
* num_layers: 18
* heads:      configs.heads = {
        'hm_cen': configs.num_classes,
        'cen_offset': configs.num_center_offset,
        'direction': configs.num_direction,
        'z_coor': configs.num_z,
        'dim': configs.num_dim
    }
    
* head_conv:  64

* configs.imagenet_pretrained = True
* configs.head_conv = 64
* configs.num_classes = 3
* configs.num_center_offset = 2
* configs.num_z = 1
* configs.num_dim = 3
* configs.num_direction = 2  # sin, cos


In [ ]:
configs['batch_size'] = 1
configs['num_workers'] = 1
heads_cfg = {'hm_cen': 3, 'cen_offset': 2, 'direction': 2, 'z_coor': 1, 'dim': 3}
model = fpn_resnet.get_pose_net(num_layers=18, heads=heads_cfg, head_conv=64, imagenet_pretrained=True)

In [ ]:
from data_process.kitti_dataloader import create_train_dataloader, create_val_dataloader, create_test_dataloader

model.eval()

test_dataloader = create_test_dataloader(configs)

with torch.no_grad():
    for batch_idx, batch_data in enumerate(test_dataloader):
        metadatas, bev_maps = batch_data
        print(batch_idx, bev_maps.size())
        outputs = model(bev_maps.float())
        
        for k in outputs:
            print(k, outputs[k].size())
        
        if batch_idx > 2:
            break
        

In [ ]:
outputs['hm_cen']

### 2. Optimizer and lr_scheduler

In [ ]:
hasattr(model, 'module')

In [ ]:
train_params = [param for param in model.parameters() if param.requires_grad]

In [ ]:
len(train_params)

In [ ]:
# learning rate and weight decay configurable, adam as default
optimizer = torch.optim.Adam(train_params, lr=0.001, weight_decay=0.0)

In [ ]:
lr_scheduler = 

In [ ]:
train_dataloader, train_sampler = create_train_dataloader(configs)

metadatas, bev, targets = None, None, None

for batch_idx, batch_data in enumerate(train_dataloader):
    metadatas, bev, targets = batch_data
    
    break


In [ ]:
targets['hm_cen'][0].size()

In [ ]:
targets['hm_cen'][0][1][93, 38]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
mat = targets['hm_cen'][0][1].numpy()

In [ ]:
plt.imshow(mat)

In [ ]:
mat[91:94, 36:40]